In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
np.random.seed(1)

true_beta = np.array([-1,1]) 


noise = 1
sigma = 10
A = np.array([[1, 0],[-1, noise]])
Sigma = A@A.T 


size  = 100
precision = 100
start = -4
end = 4
n_samples = 10
n_samples_test = 200



err_ols = 0
err_ridge = 0
err_mine = 0

for i in range(10):
    X = np.random.randn(2, n_samples)
    X = A@X 
    y = X.T @ true_beta + np.random.randn(n_samples)*sigma



    beta_ols = np.linalg.inv(X@X.T)@X@y


    X_train = X[:,:n_samples//2]
    y_train = y[:n_samples//2]
    X_val = X[:,n_samples//2:]
    y_val = y[n_samples//2:]



    # for k in [  6, 7, 8, 9]:
        # print(np.sum((y_val-X_val.T@np.linalg.inv(X_train@X_train.T+k*np.eye(2))@X_train@y_train)**2))

    k = 7
    beta_ridge = np.linalg.inv(X_val@X_val.T+k*np.eye(2))@X_val@y_val


    X_test = np.random.randn(2, n_samples_test)
    X_test = A@X_test 
    y_test = X_test.T @ true_beta + np.random.randn(n_samples_test)*sigma


    from  scipy.optimize import minimize



    def error(X_test, X_train, y_train, k):
        y_hat = X_test.T @ np.linalg.inv(X_train@X_train.T +k*np.eye(2))@X_train@y_train

        y_ols = X_test.T @ np.linalg.inv(X_train@X_train.T)@X_train@y_train

        X_hat = X_test.T @ np.linalg.inv(X_train@X_train.T +k*np.eye(2))@X_train
        X_ols = X_test.T @ np.linalg.inv(X_train@X_train.T)@X_train 

        return y_hat.T@y_hat -2*y_hat.T@y_ols +2*np.trace((X_hat.T@X_ols))



    def f(k, X):
        return error(X[0], X[1], X[2], k[0])


    # print(len(X_test.T))
    err = 0
    for i in range(len(X_test.T)):
        k0 = [1]
        X_f = [X_test[:, i:i+1], X, y]

        res = minimize(lambda k: f(k, X_f), k0, method='BFGS')
        bestk = res.x[0]
        # print(bestk)
        y_pred =X_test[:, i:i+1].T@np.linalg.inv(X@X.T +bestk*np.eye(2))@X@y


        err+= (y_pred-y_test[i])**2
        # print(f"k-met={(y_pred-y_test[i])**2}")
        # print(f"ols  ={(X_test[:, i:i+1].T@beta_ols-y_test[i])**2}")
        # print()


    err_ols +=    np.sum((y_test-X_test.T@beta_ols)**2)
    err_ridge +=np.sum((y_test-X_test.T@beta_ridge)**2)
    err_mine+=err

print(err_mine)
print(err_ridge)
print(err_ols)




[233882.54479614]
204105.46447886422
234992.16955409432


In [ ]:

      
# for k in [1.3, 1.4, 1.41, 1.5, 1.7]:
#     print(error(A, X, y, k))

# k_best =  1.41


# beta_best = np.linalg.inv(X@X.T+k_best*np.eye(2))@X@y

# print(np.sum((y_test-X_test.T@beta_best)**2))

[21589.74535018]
